In [8]:
!pip3 install -q -U google-generativeai

In [9]:
import time
from tqdm import tqdm
import json
from dotenv import load_dotenv
import os
from requests.exceptions import HTTPError
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY2")

In [10]:
import google.generativeai as genai
import pandas as pd

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

In [11]:
# Set safety settings
safety_settings = [
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}
]

In [51]:
df = pd.read_csv("labelled_csv/gemini-image-labels.csv")
df.head()

,Unnamed: 0,submissionId,submissionURL,submissionTitle,context1,context2,utterance,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?,is_1,is_2,is_3
0,0,csgp00,https://i.redd.it/z8ey88ejkeh31.jpg,dog injected full of drugs and kidnapped,it's full of doguments.,NaN,the look on the dogs face shows that even they...,1,0,1,1,1,1
1,1,bwmohn,https://i.redd.it/6roixwhh5b231.jpg,sick man cuts poor long dog in half and it fuc...,poor doggo lost his bark,post this on r/confusing_perspective,do ggo,1,0,1,0,0,0
2,2,drrqty,https://i.imgur.com/7mkUGwN.jpg,for real tho,the world's oldest teenager,bet he got a splinter here and there,fun fact: michelangelo used a unique tool he a...,0,1,1,1,1,1
3,3,uzn0h9,https://i.redd.it/s6fwn5ngu7291.jpg,welcome to america,link? i searched that website for articles wit...,"nimby, nothing new",what do people think the alternative for homel...,1,0,1,1,1,1
4,4,h9nea0,https://i.redd.it/w86h3pibo4551.jpg,you could say that the jedi's mind was truly.....,NaN,"high midichlorine count != jedi/sith, just the...","there was a comic in ledgends about this, i do...",0,1,1,1,1,1


### Initializing new columns for llm generated annotations

In [52]:
df['ts_1'] = None
df['ts_2'] = None
df['ts_3'] = None

df.head()

,Unnamed: 0,submissionId,submissionURL,submissionTitle,context1,context2,utterance,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?,is_1,is_2,is_3,ts_1,ts_2,ts_3
0,0,csgp00,https://i.redd.it/z8ey88ejkeh31.jpg,dog injected full of drugs and kidnapped,it's full of doguments.,NaN,the look on the dogs face shows that even they...,1,0,1,1,1,1,None,None,None
1,1,bwmohn,https://i.redd.it/6roixwhh5b231.jpg,sick man cuts poor long dog in half and it fuc...,poor doggo lost his bark,post this on r/confusing_perspective,do ggo,1,0,1,0,0,0,None,None,None
2,2,drrqty,https://i.imgur.com/7mkUGwN.jpg,for real tho,the world's oldest teenager,bet he got a splinter here and there,fun fact: michelangelo used a unique tool he a...,0,1,1,1,1,1,None,None,None
3,3,uzn0h9,https://i.redd.it/s6fwn5ngu7291.jpg,welcome to america,link? i searched that website for articles wit...,"nimby, nothing new",what do people think the alternative for homel...,1,0,1,1,1,1,None,None,None
4,4,h9nea0,https://i.redd.it/w86h3pibo4551.jpg,you could say that the jedi's mind was truly.....,NaN,"high midichlorine count != jedi/sith, just the...","there was a comic in ledgends about this, i do...",0,1,1,1,1,1,None,None,None


In [100]:
def provide_prompt_for_text_labels(context, utterance):
    message = """
    TASK:
    You are give a context and an utterance. Analyse the sarcasm of the utterance, and respond with one of the following labels: 0(Non-Sarcastic) or 1(Sarcastic).

    INSTRUCTIONS:
    1) Make the utterance sarcastic only when it expresses the opposite of what is implied, following the context.
    2) Make the utterance non sarcastic (0) in case of ambiguity or lack of sufficient context.
    3) Most utterance are non sarcastic.
    
    Given the following few shot examples (delimited by <examples></examples>)
    <examples>
    Context: so mr thompson, you say you have some programming skills???
    Utterance: probably referring to the "readability" certification. it's not hard, it just exempts you from needing a peer review on commits and enables you to approve other people's commits. totally overblown. i had readability in go, python, golang, and c when i worked there. you use an existing commit to request a readability certification.
    Your Response: {"Rating": "0"}
    ----

    Context: we’ve come full circle
    Utterance: there are three things you don’t discuss with strangers: religion, politics, and star wars.
    Your Response: {"Rating": "1"}
    ----

    Context: "clone wars? what are you talking about?"
    Utterance: i am one of the guys that saw star wars as a little boy (age 7, saw it 1978) the line was perfect!it immediately triggered my imagination of a dystopia where humans were cloned for being soldiers and maybe rebelled or tried to dominate which lead to a war. not far away from what was created later.
    Your Response: {"Rating": "0"}
    ----

    Context: keepers of peace, but in the most violent way
    Utterance: cut their hand off. that's the sith way. or go full obi and cut both their arms and legs off. that'll show em. bastards.
    Your Response: {"Rating": "1"}
    ----

    Context: people dont like videos cut too short.
    Utterance: then later, they would cut off just one wire from a 1,000,000 volt power grid
    Your Response: {"Rating": "0"}
    ----

    Context: roommates rice cooker has turned into brain-like tissue. he refuses to clean it, and leaves it on the shared kitchen counter.
    Utterance: that’s a biohazard.
    Your Response: {"Rating": "0"}
    ----

    Context: found a camera in my air b&b. it was halfway behind the painting with only the lens peeking out. discovered it because we heard it clicking after me and my girl got out of the shower.
    Utterance: that's not mildly infuriating, that's much worse.
    Your Response: {"Rating": "0"}
    ----

    Context: sick fuck fuses dogs togeter with black sorcery
    Utterance: i'm so confused rn
    Your Response: {"Rating": "1"}
    ----

    Context: this restaurant covered up the "no tip" option with a sticker to force tipping
    Utterance: percentage = 0%
    Your Response: {"Rating": "0"}
    ----

    Context: looks like i won't be listening to my new vinyl record. thanks, usps
    Utterance: how can an assigned delivery person not know the basic rule of don't bend the damn packages?!
    Your Response: {"Rating": "0"}
    </examples>
    ----
    Context: someone put a paper trump sticker on my sister’s car because we have a biden sign in our front yard.
    Utterance: that’s so stupid. i don’t care if you’re a trump supporter or a biden supporter, do not vandalize other peoples property because they like a different candidate then you.
    Your Response: {"Rating": "0"}
    """

    message = message + f"""
    Generate a valid JSON for the following(delimited by <test></test>)
    <test>
    Context: {context}
    Utterance: {utterance}
    Your Response: 
    </test>
    Respond only with valid JSON. Do not write an introduction or summary.
    """
    
    return message

# Defining a response schema in order to obtain consistently formatted response
response_schema_text = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "Rating": {
                "type": "integer",
            },
        },
        "required": ["Rating"],
    },
}

def detect_sarcasm_in_text(context: str, utterance: str):
    response = model.generate_content(
        provide_prompt_for_text_labels(context, utterance),
        safety_settings=safety_settings,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json", 
            response_schema=response_schema_text, 
            temperature=0
        )
    )
    return response.text

In [102]:
detect_sarcasm_in_text("This is a good view", "Take a pic")

'[{"Rating": "0"}]\n'

# Label Generation

In [103]:
curIndex = 0

In [104]:
curIndex

0

In [105]:
df.shape

(100, 16)

In [ ]:
for i in range(curIndex, df.shape[0]):
    time.sleep(4) # to deal with api limit error
    response_rating = 0
    try:
        context = df.iloc[i]['submissionTitle']
        utterance = df.iloc[i]['utterance']
        
        # Attempt to detect sarcasm in the text
        response = detect_sarcasm_in_text(context, utterance)
        
        # Parse the response and update the DataFrame
        response_as_dict = json.loads(response)
        if len(response_as_dict) > 0:
            response_rating = float(response_as_dict[0].get('Rating'))
            is_sarcastic = int(response_rating > 5)
            
    except Exception as e:
        print(e)
        is_sarcastic = 0
        
    finally:
        print(utterance)
        print("Response Rating: ", response_rating, " Original: ", df.iloc[i]['isTextSarcastic?'], "New: ", is_sarcastic)
        df.at[i, 'ts_1'] = is_sarcastic
        curIndex = i

In [106]:
for i in range(curIndex, df.shape[0]):
    time.sleep(4) # to deal with api limit error
    response_rating = 0
    try:
        context = df.iloc[i]['submissionTitle']
        utterance = df.iloc[i]['utterance']
        
        # Attempt to detect sarcasm in the text
        response = detect_sarcasm_in_text(context, utterance)
        
        # Parse the response and update the DataFrame
        response_as_dict = json.loads(response)
        if len(response_as_dict) > 0:
            response_rating = int(response_as_dict[0].get('Rating'))
            is_sarcastic = response_rating
            
    except Exception as e:
        print(e)
        is_sarcastic = 0
        
    finally:
        print(utterance)
        print("Response Rating: ", response_rating, " Original: ", df.iloc[i]['isTextSarcastic?'], "New: ", is_sarcastic)
        df.at[i, 'ts_1'] = is_sarcastic
        curIndex = i

the look on the dogs face shows that even they know that it was a shit joke.
Response Rating:  1  Original:  1 New:  1
do       ggo
Response Rating:  1  Original:  1 New:  1
fun fact: michelangelo used a unique tool he acquired in the orient to chisel is sculpture masterpieces.  https://i.imgur.com/vn1vxjt.jpg
Response Rating:  1  Original:  0 New:  1
what do people think the alternative for homeless people is???

you can put all the anti-homeless spikes/architecture you want.  they’re still homeless.
Response Rating:  1  Original:  1 New:  1
there was a comic in ledgends about this, i don't recall the name but the lore master on yt made a video about it.
Response Rating:  0  Original:  0 New:  0
i did this in quebec recently and wanted to die.

i said "por favor" instead of "s'il vous plait."
Response Rating:  1  Original:  0 New:  1
that's a dumb question.  what should happen is multiple 37 x 0.34 and round that answer to the nearest whole number.
Response Rating:  1  Original:  0 Ne

In [41]:
df['isTextSarcastic?'].value_counts()

isTextSarcastic?
1    65
0    35
Name: count, dtype: int64

In [1]:
df.head()

NameError: name 'df' is not defined

In [108]:
df['ts_1'].value_counts()

ts_1
1    95
0     5
Name: count, dtype: int64

# Evaluation

In [34]:
from sklearn.metrics import cohen_kappa_score
def compute_accuracy(y_true, y_pred):
    """
    Computes accuracy by comparing the true labels with the predicted labels.
    """
    correct_predictions = sum(y_t == y_p for y_t, y_p in zip(y_true, y_pred))
    accuracy = correct_predictions / len(y_true)
    return accuracy

def compute_precision(y_true, y_pred):
    """
    Computes precision by calculating the ratio of true positives to the sum of true and false positives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    predicted_positives = sum(y_p == 1 for y_p in y_pred)
    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    return precision

def compute_recall(y_true, y_pred):
    """
    Computes recall by calculating the ratio of true positives to the sum of true positives and false negatives.
    """
    true_positives = sum((y_t == 1 and y_p == 1) for y_t, y_p in zip(y_true, y_pred))
    actual_positives = sum(y_t == 1 for y_t in y_true)
    recall = true_positives / actual_positives if actual_positives > 0 else 0
    return recall

def compute_f1(precision, recall):
    """
    Computes F1 score as the harmonic mean of precision and recall.
    """
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def evaluate(y_true, y_pred):
    accuracy = compute_accuracy(y_true, y_pred)
    precision = compute_precision(y_true, y_pred)
    recall = compute_recall(y_true, y_pred)
    f1 = compute_f1(precision, recall)
    kappa = cohen_kappa_score(y_true, y_pred)
    print("\tAccuracy: ", accuracy, "\n", "\tPrecision: ", precision,"\n", "\tRecall: ", recall, "\n", "\tF1 score: ", f1)
    

In [96]:
evaluate(df["isTextSarcastic?"].to_list(), df["ts_3"].to_list())

	Accuracy:  0.67 
 	Precision:  0.6632653061224489 
 	Recall:  1.0 
 	F1 score:  0.7975460122699386


In [95]:
df.head()

,Unnamed: 0,submissionId,submissionURL,submissionTitle,context1,context2,utterance,isTextSarcastic?,isImageSarcastic?,isTogetherSarcastic?,is_1,is_2,is_3,ts_1,ts_2,ts_3
0,0,csgp00,https://i.redd.it/z8ey88ejkeh31.jpg,dog injected full of drugs and kidnapped,it's full of doguments.,NaN,the look on the dogs face shows that even they...,1,0,1,1,1,1,None,None,1
1,1,bwmohn,https://i.redd.it/6roixwhh5b231.jpg,sick man cuts poor long dog in half and it fuc...,poor doggo lost his bark,post this on r/confusing_perspective,do ggo,1,0,1,0,0,0,None,None,1
2,2,drrqty,https://i.imgur.com/7mkUGwN.jpg,for real tho,the world's oldest teenager,bet he got a splinter here and there,fun fact: michelangelo used a unique tool he a...,0,1,1,1,1,1,None,None,1
3,3,uzn0h9,https://i.redd.it/s6fwn5ngu7291.jpg,welcome to america,link? i searched that website for articles wit...,"nimby, nothing new",what do people think the alternative for homel...,1,0,1,1,1,1,None,None,1
4,4,h9nea0,https://i.redd.it/w86h3pibo4551.jpg,you could say that the jedi's mind was truly.....,NaN,"high midichlorine count != jedi/sith, just the...","there was a comic in ledgends about this, i do...",0,1,1,1,1,1,None,None,0
